# GIS算法

In [126]:
samples = []
Y = set([])
numXY = defaultdict(int)
filename = 'weather_data.csv'
with open(filename,'r') as f:
    f = f.readlines()[1:]
    for line in f:
        if line.index == 1:
            continue
        sample = line.strip().split(",")
        if len(sample) < 2:
            continue
        y = sample[0]
        X = sample[1:]
        samples.append(sample)
        Y.add(y)
        for x in set(X):
            numXY[(x,y)] += 1

In [127]:
samples

[['no', 'sunny', 'hot', 'high', 'FALSE'],
 ['no', 'sunny', 'hot', 'high', 'TRUE'],
 ['yes', 'overcast', 'hot', 'high', 'FALSE'],
 ['yes', 'rainy', 'mild', 'high', 'FALSE'],
 ['yes', 'rainy', 'cool', 'normal', 'FALSE'],
 ['no', 'rainy', 'cool', 'normal', 'TRUE'],
 ['yes', 'overcast', 'cool', 'normal', 'TRUE'],
 ['no', 'sunny', 'mild', 'high', 'FALSE'],
 ['yes', 'sunny', 'cool', 'normal', 'FALSE'],
 ['yes', 'rainy', 'mild', 'normal', 'FALSE'],
 ['yes', 'sunny', 'mild', 'normal', 'TRUE'],
 ['yes', 'overcast', 'mild', 'high', 'TRUE'],
 ['yes', 'overcast', 'hot', 'normal', 'FALSE'],
 ['no', 'rainy', 'mild', 'high', 'TRUE']]

In [128]:
Y

{'no', 'yes'}

In [129]:
numXY

defaultdict(int,
            {('FALSE', 'no'): 2,
             ('high', 'no'): 4,
             ('sunny', 'no'): 3,
             ('hot', 'no'): 2,
             ('TRUE', 'no'): 3,
             ('overcast', 'yes'): 4,
             ('high', 'yes'): 3,
             ('FALSE', 'yes'): 6,
             ('hot', 'yes'): 2,
             ('mild', 'yes'): 4,
             ('rainy', 'yes'): 3,
             ('cool', 'yes'): 3,
             ('normal', 'yes'): 6,
             ('cool', 'no'): 1,
             ('normal', 'no'): 1,
             ('rainy', 'no'): 2,
             ('TRUE', 'yes'): 3,
             ('mild', 'no'): 2,
             ('sunny', 'yes'): 2})

In [130]:
N = len(samples)
N

14

In [131]:
n = len(numXY)
n

19

In [132]:
C = max([len(sample)-1 for sample in samples])
C

4

In [147]:
import numpy as np
w = np.zeros(n)
w_last = w.copy()

In [148]:
ep_samples = np.zeros(n)
ep_samples

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [149]:
xyID = {}
for i,xy in enumerate(numXY):
    ep_samples[i] = numXY[xy] * 1.0 / N
    xyID[xy] = i

In [150]:
ep_samples

array([0.14285714, 0.28571429, 0.21428571, 0.14285714, 0.21428571,
       0.28571429, 0.21428571, 0.42857143, 0.14285714, 0.28571429,
       0.21428571, 0.21428571, 0.42857143, 0.07142857, 0.07142857,
       0.14285714, 0.21428571, 0.14285714, 0.14285714])

In [151]:
xyID

{('FALSE', 'no'): 0,
 ('high', 'no'): 1,
 ('sunny', 'no'): 2,
 ('hot', 'no'): 3,
 ('TRUE', 'no'): 4,
 ('overcast', 'yes'): 5,
 ('high', 'yes'): 6,
 ('FALSE', 'yes'): 7,
 ('hot', 'yes'): 8,
 ('mild', 'yes'): 9,
 ('rainy', 'yes'): 10,
 ('cool', 'yes'): 11,
 ('normal', 'yes'): 12,
 ('cool', 'no'): 13,
 ('normal', 'no'): 14,
 ('rainy', 'no'): 15,
 ('TRUE', 'yes'): 16,
 ('mild', 'no'): 17,
 ('sunny', 'yes'): 18}

In [157]:
def zx_func(X):
    ZX = 0.0
    for y in Y:
        sum = 0.0
        for x in X:
            if(x,y) in numXY:
                sum += w[xyID[(x,y)]]
        ZX += math.exp(sum)
    return ZX

In [158]:
[zx_func(sample[1:]) for sample in samples]

[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0]

In [159]:
def p_func(X):
    zx = zx_func(X)
    result_p = []
    for y in Y:
        sum = 0.0
        for x in X:
            if (x,y) in numXY:
                sum += w[xyID[(x,y)]]
        p = 1.0 / zx * math.exp(sum)
        result_p.append((y,p))
    return result_p

In [160]:
[p_func(sample[1:]) for sample in samples]

[[('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)],
 [('no', 0.5), ('yes', 0.5)]]

In [161]:
w = np.zeros(n)
w_last = w.copy()
for i in range(1000):
    w_last = w.copy()
    ep = np.zeros(n)
    for sample in samples:
        X = sample[1:]
        pyx = p_func(X)
        for y,p in pyx:
            for x in X:
                if (x,y) in numXY:
                    ep[xyID[(x,y)]] += p * 1.0 / N
    for i,x in enumerate(w):
        w[i] += 1.0 / C * math.log(ep_samples[i] / ep[i])
    if max([w_last_i-w_i for (w_last_i,w_i) in zip(w_last,w)]) < 0.01:
        break

In [162]:
w

array([-3.09636514,  1.25174271,  2.80360494,  0.10571373,  1.33438423,
        3.9918132 , -1.59514366,  1.13207234, -0.09397029,  1.40543437,
       -1.34380257, -0.84370095,  1.3137177 ,  2.783914  , -6.70804184,
        2.171492  , -1.2808014 , -2.63112494, -3.85569374])

In [166]:
predict_strings = ["sunny,hot,high,FALSE","overcast,hot,high,FALSE","sunny,cool,high,TRUE"]
predict_Xs = [predict_string.split(',') for predict_string in predict_strings]
[p_func(X) for X in predict_Xs]

[[('no', 0.9958373481280207), ('yes', 0.0041626518719793)],
 [('no', 0.00563178976395537), ('yes', 0.9943682102360447)],
 [('no', 0.9999998553553482), ('yes', 1.4464465173635744e-07)]]

# IIS算法